# Trigger delays analysis

In [8]:
%load_ext autoreload
%load_ext usnm2p.skip_kernel_extension
%autoreload 2

### Imports

In [9]:
import os 
from tqdm import tqdm

from usnm2p.bruker_utils import *
from usnm2p.fileops import *
from usnm2p.logger import logger
from usnm2p.constants import *

### Inputs

In [22]:
dataroot = '/Users/tlemaire/Documents/data/usnm/raw_bruker'
analysis = 'main'
mouseline = 'line3'

### Define function to extract delay

In [10]:
def get_trigger_delay_per_trial(fpath):
    _, _, seq_frames, seq_voutputs = parse_bruker_XML(fpath, simplify=False)
    times = pd.concat({
        Label.TRIG: seq_voutputs['absoluteTime'],
        Label.FRAME: seq_frames.loc[seq_voutputs.index, 'absoluteTime'].groupby('sequence').first(),
    }, axis=1)
    times[Label.DELAY] = times[Label.FRAME] - times[Label.TRIG]
    times.index = pd.Index(np.arange(len(times)), name= Label.TRIAL)
    return times

### Extract trigger delays for mouseline and analysis type

In [ ]:
# List datasets
mouselinedir = os.path.join(dataroot, analysis, mouseline)
datasets = [k for k in os.listdir(mouselinedir) if k != '.DS_Store']

# Extarct trigger data
mouseline_data = {}
for dataset in datasets:
    logger.info(f'extracting trigger delays from {dataset} XML acquisition files')
    acqdir = os.path.join(mouselinedir, dataset)
    acqfolders = get_data_folders(acqdir, recursive=False, exclude_patterns=['mouse'])
    acqdata = {}
    for f in tqdm(acqfolders):
        xml = os.path.join(f, get_bruker_XML(f))
        acqdata[os.path.basename(f)] = get_trigger_delay_per_trial(xml)
    acqdata = pd.concat(acqdata, names=['acquisition'])
    mouseline_data[dataset] = acqdata
mouseline_data = pd.concat(mouseline_data, names=[Label.DATASET])
mouseline_data

 2025/07/11 13:57:09: extracting trigger delays from 20191112_mouse12_region1 XML acquisition files
 2025/07/11 13:57:09: searching through /Users/tlemaire/Documents/data/usnm/raw_bruker/main/line3/20191112_mouse12_region1
 2025/07/11 13:57:09: found 16 folders containing TIF files
 2025/07/11 13:57:09: 15 folders remain after filtering


100%|██████████| 15/15 [00:02<00:00,  6.48it/s]


trigger  \
dataset                  acquisition                                       trial            
20191112_mouse12_region1 line3_100frames_100Hz_200ms_3.56Hz_08MPA_50DC-855 0        2.984   
                                                                           1       31.722   
                                                                           2        60.51   
                                                                           3       89.308   
                                                                           4      118.104   
...                                                                                   ...   
                         line3_100frames_100Hz_200ms_3.56Hz_02MPA_50DC-866 11     319.615   
                                                                           12     348.495   
                                                                           13     377.288   
                                                                           14     406.096   
                                                                           15     434.916   

                                                                                    frame  \
dataset                  acquisition                                       trial            
20191112_mouse12_region1 line3_100frames_100Hz_200ms_3.56Hz_08MPA_50DC-855 0        3.051   
                                                                           1       31.801   
                                                                           2       60.611   
                                                                           3       89.410   
                                                                           4      118.200   
...                                                                                   ...   
                         line3_100frames_100Hz_200ms_3.56Hz_02MPA_50DC-866 11     319.728   
                                                                           12     348.617   
                                                                           13     377.397   
                                                                           14     406.207   
                                                                           15     435.036   

                                                                                  delay  
dataset                  acquisition                                       trial         
20191112_mouse12_region1 line3_100frames_100Hz_200ms_3.56Hz_08MPA_50DC-855 0      0.067  
                                                                           1      0.079  
                                                                           2      0.101  
                                                                           3      0.102  
                                                                           4      0.096  
...                                                                                 ...  
                         line3_100frames_100Hz_200ms_3.56Hz_02MPA_50DC-866 11     0.113  
                                                                           12     0.122  
                                                                           13     0.109  
                                                                           14     0.111  
                                                                           15      0.12  

[240 rows x 3 columns]

### Show trigger delay stats per dataset

In [25]:
mouseline_data['delay'].groupby(Label.DATASET).agg(['mean', 'std'])

,mean,std
dataset,,
20191112_mouse12_region1,0.106867,0.013932
